In [20]:
%run nc_underestimator-mutid-ipopt.ipynb

PID Problem

In [21]:
import pyomo.environ as pyo

def build_pid_model(
    T=50,                 # 时间步数
    h=0.1,                # 步长 Δt
    scen=None,            # 单个场景数据: {"Ku":..., "tau":..., "d":[...], "sp":[...]}
    weights=(1.0, 0.01),  # 目标权重 (w_e, w_u)
    bounds={"x":(-20,20), "u":(-20,20), "Kp":(0,100), "Ki":(0,1000), "Kd":(0,1000)},
    use_cvar=False,       # 保留参数，但单场景下一般不需要
    alpha=0.95
):
    """
    单场景 PID 控制模型:
      tau * x_dot + x = Ku * u + d
      u = Kp*e + Ki*I + Kd*de/dt,   e = sp - x,   I_dot = e
    离散化: 隐式欧拉
    """
    assert scen is not None, "请提供一个场景字典"

    Ku, tau, d, sp = scen["Ku"], scen["tau"], scen["d"], scen["sp"]

    m = pyo.ConcreteModel()
    m.T = pyo.RangeSet(0, T)
    m.Tm = pyo.RangeSet(1, T)

    # 一阶段变量 (PID 参数)
    m.Kp = pyo.Var(bounds=bounds["Kp"])
    m.Ki = pyo.Var(bounds=bounds["Ki"])
    m.Kd = pyo.Var(bounds=bounds["Kd"])

    # 二阶段变量
    m.x = pyo.Var(m.T, bounds=bounds["x"])
    m.u = pyo.Var(m.T, bounds=bounds["u"])
    m.e = pyo.Var(m.T)
    m.I = pyo.Var(m.T)

    # 误差 e_t = sp_t - x_t
    def _err_rule(m, t): return m.e[t] == sp[t] - m.x[t]
    m.err_def = pyo.Constraint(m.T, rule=_err_rule)

    # I 动态
    def _I_dyn(m, t): return m.I[t] == m.I[t-1] + h*m.e[t]
    m.I_dyn = pyo.Constraint(m.Tm, rule=_I_dyn)

    # 系统动态 (隐式欧拉)
    def _x_dyn(m, t):
        return m.x[t] == m.x[t-1] + (h/tau)*(-m.x[t] + Ku*m.u[t] + d[t])
    m.x_dyn = pyo.Constraint(m.Tm, rule=_x_dyn)

    # PID 控制律
    def _pid_rule(m, t):
        if t == 0:
            return m.u[t] == m.Kp*m.e[t] + m.Ki*m.I[t]
        return m.u[t] == m.Kp*m.e[t] + m.Ki*m.I[t] + m.Kd*(m.e[t]-m.e[t-1])/h
    m.pid = pyo.Constraint(m.T, rule=_pid_rule)

    # 初值
    m.x0 = pyo.Constraint(expr=m.x[0] == 0)
    m.I0 = pyo.Constraint(expr=m.I[0] == 0)

    # 成本函数 (单场景就是一个值)
    w_e, w_u = weights
    m.cost = pyo.Expression(expr=sum(
        h*(w_e*m.e[t]**2 + w_u*m.u[t]**2) for t in m.T
    ))

    # 目标: 单场景情况下就直接最小化 cost
    #m.obj = pyo.Objective(expr=m.cost, sense=pyo.minimize)
    m.obj_expr = pyo.Expression(expr = m.cost)

    return m,  [m.Kp, m.Ki, m.Kd]


In [22]:
T, h = 60, 0.1
times = [t for t in range(T+1)]

def step_sp(t): return 1.0 if t*h >= 0.5 else 0.0
def make_d(amp): return [amp*math.sin(0.5*t*h) for t in times]
scenarios = {
    1: {"prob": 0.4, "Ku": 3.0, "tau": 2.0, "d": make_d(0.2), "sp": [step_sp(t) for t in times]},
}
'''scenarios = {
    1: {"prob": 0.4, "Ku": 3.0, "tau": 2.0, "d": make_d(0.2), "sp": [step_sp(t) for t in times]},
    2: {"prob": 0.4, "Ku": 2.7, "tau": 1.8, "d": make_d(0.5), "sp": [step_sp(t) for t in times]},
    3: {"prob": 0.2, "Ku": 3.3, "tau": 2.2, "d": make_d(0.8), "sp": [step_sp(t) for t in times]},
}'''
'''scen = {
    "Ku": 3.0,
    "tau": 2.0,
    "d": make_d(0.3),
    "sp": [step_sp(t) for t in times],
}'''
model_list = []
first_stg_vars_list = []
for _, scen in scenarios.items():
    m, m_f_stg_vars = build_pid_model(T=T, h=h, scen=scen)
    model_list.append(m)
    first_stg_vars_list.append(m_f_stg_vars)
bounds={"x":(-20,20), "u":(-20,20), "Kp":(0,100), "Ki":(0,1000), "Kd":(0,1000)}
m_tmpl = pyo.ConcreteModel()
m_tmpl.T = pyo.RangeSet(0, T)
m_tmpl.Tm = pyo.RangeSet(1, T)
m_tmpl.Kp = pyo.Var(bounds=bounds["Kp"])
m_tmpl.Ki = pyo.Var(bounds=bounds["Ki"])
m_tmpl.Kd = pyo.Var(bounds=bounds["Kd"])
m_tmpl_list = [m_tmpl, [m_tmpl.Kp, m_tmpl.Ki, m_tmpl.Kd]]

In [23]:
nc_underest(model_list, first_stg_vars_list, m_tmpl_list, target_nodes=10,
                picture_shown=False, v_list=False, tolerance=1e-8
)

corner nodes are  [(0.0, 0.0, 0.0), (0.0, 0.0, 1000.0), (0.0, 1000.0, 0.0), (0.0, 1000.0, 1000.0), (100.0, 0.0, 0.0), (100.0, 0.0, 1000.0), (100.0, 1000.0, 0.0), (100.0, 1000.0, 1000.0)]
as_nodes_list are  [[4.540235074015775, 0.05320473411804464, 0.048865289110605704, 0.0532164212240754, 0.04795114717129116, 0.05320592569636047, 0.05042763872461542, 0.05321691343179907]]
Start from  8  nodes
The goal is to get  10  nodes
##################################################
##################################################
Start adding node  9
 
Solving scenario  0
new node is  (0.6156949844910186, 6.156978352769934, 6.156978625327275)
ms is  -4.448494875995676
OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
True
0
True
1000
True
1000
As_min at possible new node is -inf
Real value at possible new node is 0.04795219574564336
error at possible new node is inf
Sum *****************************************
error at y_star is  inf
y_star is  (99.99900401233101, 0.0112007720954449

(-inf,
 [(0.0, 0.0, 0.0),
  (0.0, 0.0, 1000.0),
  (0.0, 1000.0, 0.0),
  (0.0, 1000.0, 1000.0),
  (100.0, 0.0, 0.0),
  (100.0, 0.0, 1000.0),
  (100.0, 1000.0, 0.0),
  (100.0, 1000.0, 1000.0),
  (99.99900401233101, 0.011200772095444927, 0.0026488148954376123),
  (99.99915132755807, 0.009636430994860943, 0.0025307281968381524)],
 [[9, 10],
  [-inf, -inf],
  [(99.99900401233101, 0.011200772095444927, 0.0026488148954376123),
   (99.99915132755807, 0.009636430994860943, 0.0025307281968381524)]])

In [ ]:
        print('OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO')
        print(model_sum.Kp.has_lb())   # 是否有下界
        print(model_sum.Kp.lb)         # 下界是多少（None 表示没有）
        print(model_sum.Ki.has_ub())   # 是否有上界
        print(model_sum.Ki.ub)         # 上界是多少（None 表示没有）
        print(model_sum.Kd.has_ub())   # 是否有上界
        print(model_sum.Kd.ub)         # 上界是多少（None 表示没有）